In [14]:
!pip install folium
!pip install pandas
import folium
import pandas as pd


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [16]:
# Download and read the `spacex_launch_geo.csv`
import pandas as pd
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
#resp = await fetch(URL)
#spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(URL)

In [17]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [18]:
## Task 1: Mark all launch sites on a map

# Download and read the `spacex_launch_geo.csv`
#from js import fetch
#import io
#URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
#resp = await fetch(URL)
#spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
#spacex_df=pd.read_csv(spacex_csv_file)

#convert to python--no js
import requests
from io import StringIO
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = requests.get(URL)
if resp.status_code == 200:
    spacex_df = pd.read_csv(StringIO(resp.text))




# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df
#spacex_df.head(20)

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

# Add all launch sites to the map
for index, row in launch_sites_df.iterrows():
    # Get coordinates
    launch_site_coord = [row['Lat'], row['Long']]

    # Add circle
    circle = folium.Circle(launch_site_coord, radius=1000, color='blue', fill=True).add_child(folium.Popup(row['Launch Site']))

    # Add marker with the launch site name
    marker = folium.map.Marker(
        launch_site_coord,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:blue;"><b>%s</b></div>' % row['Launch Site'],
        )
    )

    # Add circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


In [19]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Loop through launch_sites_df to add circles for each launch site
for index, row in launch_sites_df.iterrows():
    # Get coordinates
    launch_site_coord = [row['Lat'], row['Long']]

    # Add circle
    circle = folium.Circle(launch_site_coord, radius=1000, color='blue', fill=True).add_child(folium.Popup(row['Launch Site']))

    # Add marker with the launch site name
    marker = folium.map.Marker(
        launch_site_coord,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 24; color:red;"><b>%s</b></div>' % row['Launch Site'],
        )
    )

    # Add circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

In [20]:
# Task 2: Mark the success/failed launches for each site on the map

#SEE BELOW

In [21]:
spacex_df.tail(10)
len(spacex_df)

56

In [22]:
marker_cluster = MarkerCluster()

In [23]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

#SEE BELOW

In [24]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
for index, record in spacex_df.iterrows():

    launch_site_coord = [record['Lat'], record['Long']]

    # Determine marker color based on 'class' column
    marker_color = 'green' if record['class'] == 1 else 'red'

    # create a Marker object with its coordinate
    marker = folium.Marker(
        location=launch_site_coord,
        #icon=folium.Icon(color='white', icon_color=row['marker_color']
        icon=folium.Icon(color=marker_color),
        # and customize the Marker's icon property to indicate if this launch was successed or failed,
        popup=f"Launch Site: {record['Launch Site']}\nOutcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )

    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Display the map
site_map

In [25]:
# TASK 3: Calculate the distances between a launch site to its proximities

#See Below


In [26]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [27]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [28]:
# find coordinate of the closet coastline
coordinate = [28.56332, -80.56796]
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_lat = 28.56345
launch_site_lon = -80.57678
coastline_lat = 28.56332
coastline_lon = -80.56796

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance = distance_coastline

In [29]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
# for example
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
       )
   )
site_map.add_child(distance_marker)

In [30]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
launch_site =[launch_site_lat, launch_site_lon]
coastline = [coastline_lat, coastline_lon]
coordinates = [launch_site, coastline ]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [31]:
# Create a marker with distance to a closest city, railway, highway, etc.
launch_site_coord = [28.56345, -80.57678]
NASA_RR = [28.5712, -80.58542]
Parkway = [28.56373, -80.57079]
Titusville = [28.61045, -80.81543]

# Calculate distances
distance_to_rr = calculate_distance(28.56345, 28.5712,-80.57678, -80.58542)
distance_to_parkway = calculate_distance(28.56345, 28.56373, -80.57678, -80.57079)
distance_to_titusville = calculate_distance(28.56345, 28.61045, -80.57678, -80.81543)

#NASA RR marker
distance_marker_rr = folium.Marker(
   location= NASA_RR,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_rr),
       )
   )
site_map.add_child(distance_marker_rr)

# Parkway marker
distance_marker_parkway = folium.Marker(
    location=Parkway,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_parkway),
    )
)
site_map.add_child(distance_marker_parkway)

# Titusville marker
distance_marker_titusville = folium.Marker(
    location=Titusville,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_titusville),
    )
)
site_map.add_child(distance_marker_rr)
site_map.add_child(distance_marker_parkway)
site_map.add_child(distance_marker_titusville)



# Draw a line between the marker to the launch site

launch_site =[28.56345, -80.57678]
NASA_RR = [28.5712, -80.58542]
coordinate1 = [launch_site, NASA_RR ]
lines=folium.PolyLine(locations=coordinate1, weight=1)

launch_site =[28.56345, -80.57678]
Parkway = [28.56373, -80.57079]
coordinate2 = [launch_site, Parkway]
lines=folium.PolyLine(locations=coordinate2, weight=1)

launch_site =[28.56345, -80.57678]
Titusville = [28.61045, -80.81543]
coordinate3 = [launch_site, Titusville ]
lines=folium.PolyLine(locations=coordinate3, weight=1)

site_map.add_child(lines)

In [32]:
#yes, lines added to nearest coastline, railway, highway and city!!! 0.83Km from Coastline
# see plot to Titusville

site_map.save("site_map.html")